# Set Membership

The cell below defines two **abstract classes**: the first represents a set and basic insert/search operations on it. You will need to impement this API four times, to implement (1) sequential search, (2) binary search tree, (3) balanced search tree, and (4) bloom filter. The second defines the synthetic data generator you will need to implement as part of your experimental framework. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [2]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

# abstract class to represent a set and its insert/search operations
class AbstractSet(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # inserts "element" in the set
    # returns "True" after successful insertion, "False" if the element is already in the set
    # element : str
    # inserted : bool
    @abstractmethod
    def insertElement(self, element):     
        inserted = False
        return inserted   
    
    # checks whether "element" is in the set
    # returns "True" if it is, "False" otherwise
    # element : str
    # found : bool
    @abstractmethod
    def searchElement(self, element):
        found = False
        return found    
    
    
    
# abstract class to represent a synthetic data generator
class AbstractTestDataGenerator(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # creates and returns a list of length "size" of strings
    # size : int
    # data : list<str>
    @abstractmethod
    def generateData(self, size):     
        data = [""]*size
        return data   


Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [ ]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE
class Node:
    def __init__(self, val):
        self.val = val
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def show(self):
        current = self.head
        while(current is not None):
            print(current.val)
            current = current.next    

Use the cell below to implement the requested API by means of **sequential search**.

In [ ]:
class SequentialSearchSet(AbstractSet):
    
    def __init__(self):   
        self.set = LinkedList()
        
    def insertElement(self, element):
        inserted = False
        current = self.set.head

        # case 1: insert to empty linkedlist
        if current is None:
            self.set.head = Node(element)
        # case 2: insert to head of linkedlist
        elif current.val > element:
            newNode = Node(element)
            newNode.next = self.set.head
            self.set.head = newNode
        # case 3: insert to somewhere else in the linkedlist
        else:
            while(current.next is not None and current.next.val <= element):
                current = current.next
            if current.val == element:
                return inserted
            else:
                newNode = Node(element)
                newNode.next = current.next
                current.next = newNode

        inserted = True
        return inserted


    def searchElement(self, element):     
        found = False
        if self.set is None:
            return found
        else:
            current = self.set.head
            while(current is not None):
                if current.val == element:
                    found = True
                    return found
                current = current.next

        return found    

Dash: just for testing so i'll remove these cells later

In [ ]:
f = open("test2-warpeace.txt", "r")
test = SequentialSearchSet()
for line in f:
    for word in line.split():
        test.insertElement(word)
test.set.show()
f.close()

In [ ]:
f = open("test-search.txt", "r")
for word in f:
    target = word.strip()
    if(test.searchElement(target)):
        print(target + " ✓")
    else:
        print(target + " ✕")

Use the cell below to implement the requested API by means of **binary search tree**.

In [ ]:
class Node:
    def __init__(self, element):
        self.element = element
        self.left = None
        self.right = None

class BinarySearchTreeSet():

    def __init__(self):
        self.root = None

    def insertElement(self, element):
        inserted = False
        if self.root == None:
            self.root = Node(element)
        else:
            current = self.root
            while 1:
                if element < current.element:
                    if current.left:
                        current = current.left
                    else:
                        current.left = Node(element)
                        break
                elif element > current.element:
                    if current.right:
                        current = current.right
                    else:
                        current.right = Node(element)
                        break
                else:
                    break
        inserted = True
        return inserted

    def searchElement(self, element):
        if self.root == None:
            return False
        else:
            current = self.root
            while current != None:
                if element == current.element:
                    return True
                elif element < current.element:
                    current = current.left
                else:
                    current = current.right
            return False

Use the cell below to implement the requested API by means of **balanced search tree**.

# Red Black Balanced Search Tree
-------------------------------------

The `RBTree` class provides an implementation of a Red Black tree with an `insert` method to add new nodes. The class also contains several helper methods to fix the tree if necessary after a new node is inserted.

The `insertElement` method works as follows:

1. If the tree is empty, the new node is inserted as the root and coloured black.
2. Otherwise, the tree is searched for the proper location to insert the new node, following the rules of a binary search tree.
3. After inserting the new node, the `_fix_tree` method is called to ensure that the Red Black tree properties are maintained.

The `_fix_tree` method is responsible for checking and correcting any violations of the Red Black tree properties. It follows a series of cases to determine what rotation(s) and colour changes, if any, are necessary to fix the tree.

The `_rotate_left` and `_rotate_right` methods are helper methods used by `_fix_tree` to perform left and right rotations, respectively, on the tree.

Together, these methods provide a working implementation of a Red Black tree with an insert method that maintains the Red Black tree properties.

In [4]:
RED = True
BLACK = False

# Node class
class RBNode: # DID NOT ADD TO MAIN NODE CLASS AS THIS REQUIRES MORE ATTRIBUTES
    def __init__(self, value, colour=RED):
        self.value = value
        self.colour = colour
        self.left = None # left child
        self.right = None # right child
        self.parent = None

    def __repr__(self):
        return f"{self.value}({self.colour})"

    def __iter__(self):
        if self.left:
            yield from self.left
        yield self.value
        if self.right:
            yield from self.right

    def grandparent(self):
        if self.parent:
            return self.parent.parent

    def uncle(self):
        if self.grandparent():
            if self.grandparent().left == self.parent:
                return self.grandparent().right
            else:
                return self.grandparent().left

    def sibling(self):
        if self.parent:
            if self.parent.left == self:
                return self.parent.right
            else:
                return self.parent.left


# Red Black Tree class
class BalancedSearchTreeSet(AbstractSet):
    
    def __init__(self):
        self.root = None

    def __repr__(self):
        return str([v for v in self])

    def __iter__(self):
        if self.root:
            yield from self.root

    def __contains__(self, value):
        if self.root is None:
            return False
        else:
            current = self.root
            while True:
                if value == current.value:
                    return True
                elif value < current.value:
                    if current.left is None:
                        return False
                    else:
                        current = current.left
                else:
                    if current.right is None:
                        return False
                    else:
                        current = current.right

        
    def insertElement(self, element):
        inserted = True

        node = RBNode(element)
        if self.root is None:
            self.root = node
            self.root.colour = BLACK
        else:
            current = self.root
            while True:
                if node.value < current.value:
                    if current.left is None:
                        current.left = node
                        node.parent = current
                        break
                    else:
                        current = current.left

                elif node.value == current.value:
                    inserted = False
                    break

                else:
                    if current.right is None:
                        current.right = node
                        node.parent = current
                        break
                    else:
                        current = current.right

            self._fixup(node)
        
        return inserted
    
    def _fixup(self, node : RBNode):
        # case 1: node is root
        if node.parent is None:
            node.colour = BLACK
            return

        # case 2: parent is black
        if node.parent.colour == BLACK:
            return

        # case 3: parent and uncle are red
        if node.uncle() and node.uncle().colour == RED:
            node.parent.colour = BLACK
            node.uncle().colour = BLACK
            node.grandparent().colour = RED
            self._fixup(node.grandparent())
            return

        # case 4: parent is red, uncle is black
        else:
            # case 4a: node is right child, parent is left child
            if node.parent.right == node and node.grandparent().left == node.parent:
                self._rotate_left(node.parent)
                node = node.left

            # case 4b: node is left child, parent is right child
            elif node.parent.left == node and node.grandparent().right == node.parent:
                self._rotate_right(node.parent)
                node = node.right

            # case 5: node is left child, parent is left child or node is right child, parent is right child
            node.parent.colour = BLACK
            node.grandparent().colour = RED
            if node.parent.left == node and node.grandparent().left == node.parent:
                self._rotate_right(node.grandparent())
            else:
                self._rotate_left(node.grandparent())


    def _rotate_left(self, node: RBNode):
        new_parent = node.right
        node.right = new_parent.left
        if new_parent.left:
            new_parent.left.parent = node
        new_parent.parent = node.parent
        if node.parent is None:
            self.root = new_parent
        elif node == node.parent.left:
            node.parent.left = new_parent
        else:
            node.parent.right = new_parent
        new_parent.left = node
        node.parent = new_parent

    def _rotate_right(self, node: RBNode):
        new_parent = node.left
        node.left = new_parent.right
        if new_parent.right:
            new_parent.right.parent = node
        new_parent.parent = node.parent
        if node.parent is None:
            self.root = new_parent
        elif node == node.parent.left:
            node.parent.left = new_parent
        else:
            node.parent.right = new_parent
        new_parent.right = node
        node.parent = new_parent

    def searchElement(self, element):     
        return element in self

**Testing Balanced Search Tree**

In [5]:
with open("test2-warpeace.txt", "r") as f:
    tree = BalancedSearchTreeSet()
    for line in f:
        for word in line.split():
            tree.insertElement(word)

with open("test-search.txt", "r") as f:
    for word in f:
        target = word.strip()
        if tree.searchElement(target):
            print(target + " ✓")
        else:
            print(target + " ✕")

a's ✓
able ✓
about ✓
above ✓
according ✓
accordingly ✓
across ✓
actually ✓
after ✓
afterwards ✓
again ✓
against ✓
ain't ✕
all ✓
allow ✓
allows ✓
almost ✓
alone ✓
along ✓
already ✓
also ✓
although ✓
always ✓
am ✓
among ✓
amongst ✓
an ✓
and ✓
another ✓
any ✓
anybody ✓
anyhow ✓
anyone ✓
anything ✓
anyway ✓
anyways ✕
anywhere ✓
apart ✓
appear ✓
appreciate ✓
appropriate ✓
are ✓
aren't ✓
around ✓
as ✓
aside ✓
ask ✓
asking ✓
associated ✓
at ✓
available ✓
away ✓
awfully ✓
be ✓
became ✓
because ✓
become ✓
becomes ✓
becoming ✓
been ✓
before ✓
beforehand ✓
behind ✓
being ✓
believe ✓
below ✓
beside ✓
besides ✓
best ✓
better ✓
between ✓
beyond ✓
both ✓
brief ✓
but ✓
by ✓
c'mon ✕
c's ✕
came ✓
can ✓
can't ✓
cannot ✓
cant ✕
cause ✓
causes ✓
certain ✓
certainly ✓
changes ✓
clearly ✓
co ✕
com ✕
come ✓
comes ✓
concerning ✓
consequently ✓
consider ✓
considering ✓
contain ✓
containing ✓
contains ✓
corresponding ✓
could ✓
couldn't ✓
course ✓
currently ✕
definitely ✓
described ✓
despite ✓
did ✓
didn't ✓
diff

Use the cell below to implement the requested API by means of **bloom filter**.

In [7]:
from bitarray import bitarray

# calculate natural logarithm
def ln(x):
    n = 1000.0
    return n * ((x ** (1/n)) - 1)

# get size of bitarray
def get_size(num_of_items, fp_prob):
        size = -(num_of_items * ln(fp_prob)) / (ln(2) ** 2)
        return int(size)

# get number of hash function to be used
def get_hash_num(bitarray_size, num_of_items):
    hash_num = (bitarray_size / num_of_items) * ln(2)
    return int(hash_num)

# Murmurhash3 32bits version
# reference: https://huagetai.github.io/posts/fcfde8ff/
def murmurhash_32(key, seed=0, blocksize=64):
    h = seed
    constant1 = 0xcc9e2d51
    constant2 = 0x1b873593
    r1 = 15
    r2 = 13
    m = 5
    n = 0xe6546b64
    bits_32 = 0xffffffff
    # Divide key into chunks of blocksize bits (default 64 bits) and iterate over each chunk
    for chunk in [key[i:i+blocksize//8] for i in range(0, len(key), blocksize//8)]:
        k = 0
        for i, c in enumerate(chunk):
            # Convert chunk into integer k by shifting bytes left by multiples of 8 bits
            k |= ord(c) << (8 * i)
        k *= constant1
        k &= bits_32  # Truncate k to 32 bits
        k = (k << r1) | (k >> 32 - r1)  # Rotate k left by 15 bits and bitwise OR with k rotated right by 17 bits
        k *= constant2
        k &= bits_32
        h ^= k  # XOR
        h = (h << r2) | (h >> 32 - r2)
        h = h * m + n
        h &= bits_32
    h ^= len(key)
    h &= bits_32
    h ^= h >> 16
    h *= 0x85ebca6b
    h &= bits_32
    h ^= h >> 13
    h *= 0xc2b2ae35
    h &= bits_32
    h ^= h >> 16
    h &= bits_32
    return h



class BloomFilterSet(AbstractSet):
    
    def __init__(self, num_of_items=80000, fp_prob=0.05):
        # ADD YOUR CODE HERE
        self.size = get_size(num_of_items, fp_prob)
        self.hash_num = get_hash_num(self.size, num_of_items)
        self.bit_array = bitarray(self.size)
        self.bit_array.setall(0)




    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
        for i in range(self.hash_num):
            val_of_hash = murmurhash_32(element, i) % self.size
            if self.bit_array[val_of_hash] == 0:
                inserted = True
                self.bit_array[val_of_hash] = 1
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE
        for i in range(self.hash_num):
            val_of_hash = murmurhash_32(element, i) % self.size
            if self.bit_array[val_of_hash] == 0:
                return found
        found = True
        return found

In [8]:
with open("test2-warpeace.txt", "r") as f:
    bf = BloomFilterSet()
    conflict = 0
    total_num = 0
    for line in f:
        for word in line.split():
            bf.insertElement(word)

In [9]:
with open("test-search.txt", "r") as f:
    for word in f:
        target = word.strip()
        if bf.searchElement(target):
            print(target + " ✓")
        else:
            print(target + " ✕")

a's ✓
able ✓
about ✓
above ✓
according ✓
accordingly ✓
across ✓
actually ✓
after ✓
afterwards ✓
again ✓
against ✓
ain't ✕
all ✓
allow ✓
allows ✓
almost ✓
alone ✓
along ✓
already ✓
also ✓
although ✓
always ✓
am ✓
among ✓
amongst ✓
an ✓
and ✓
another ✓
any ✓
anybody ✓
anyhow ✓
anyone ✓
anything ✓
anyway ✓
anyways ✕
anywhere ✓
apart ✓
appear ✓
appreciate ✓
appropriate ✓
are ✓
aren't ✓
around ✓
as ✓
aside ✓
ask ✓
asking ✓
associated ✓
at ✓
available ✓
away ✓
awfully ✓
be ✓
became ✓
because ✓
become ✓
becomes ✓
becoming ✓
been ✓
before ✓
beforehand ✓
behind ✓
being ✓
believe ✓
below ✓
beside ✓
besides ✓
best ✓
better ✓
between ✓
beyond ✓
both ✓
brief ✓
but ✓
by ✓
c'mon ✕
c's ✕
came ✓
can ✓
can't ✓
cannot ✓
cant ✕
cause ✓
causes ✓
certain ✓
certainly ✓
changes ✓
clearly ✓
co ✕
com ✕
come ✓
comes ✓
concerning ✓
consequently ✓
consider ✓
considering ✓
contain ✓
containing ✓
contains ✓
corresponding ✓
could ✓
couldn't ✓
course ✓
currently ✕
definitely ✓
described ✓
despite ✓
did ✓
didn't ✓
diff

In [ ]:
exist = []
correct = []
with open("test-search.txt", "r") as f:
    for word in f:
        target = word.strip()
        if bf.searchElement(target):
            exist.append(target)
with open("test2-warpeace.txt", "r") as f:
    for line in f:
        for item in exist:
            if item not in correct:
                if line.find(item) != -1:
                    correct.append(item)
print("False positive probability: ", 1 - len(correct) / len(exist))

Use the cell below to implement the **synthetic data generator** as part of your experimental framework.

In [ ]:
import string
import random

class TestDataGenerator(AbstractTestDataGenerator):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        
        
        pass           
        
    def generateData(self, size):     
        # ADD YOUR CODE HERE
        data = [""]*size
        

        return data   



Use the cells below for the python code needed to **fully evaluate your implementations**, first on real data and subsequently on synthetic data (i.e., read data from test files / generate synthetic one, instantiate each of the 4 set implementations in turn, then thorouhgly experiment with insert/search operations and measure their performance).

In [ ]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON REAL DATA





In [ ]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON SYNTHETIC DATA



